# Web Scraping Tripadvisor for fetching reviews for Hotel ID from Expedia  

** We need chromedriver for fetching hotel reviews from the initial URL and then use the reviews for adding to Expedia**  

In [ ]:
import re
import time, sys
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
import codecs
import os.path
import math

In [4]:
currentWD = os.getcwd()
currentWD

'C:\\Users\\Jay Padhya\\Documents\\Python_FinalProject'

In [9]:
url = 'http://www.tripadvisor.com/Hotels-g45963-Las_Vegas_Nevada-Hotels.html'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)

In [10]:
baseDir = currentWD+"\\Data"
completeDir = currentWD+"\\BackupData\\Perfect"

**Define the URL, hotelreview list and functions for getting reviews, getting hyperlinkks for further pages and processing hotels** 

In [ ]:
prevPage = url
prevPageNo = 1
hotelReviewUrlLst = []

if not os.path.exists(baseDir):
   os.makedirs(os.path.dirname(baseDir))

# gets the review count of the hotel
def getReviews(reviewLink):
    words = reviewLink.split(">") 
    review = words[-1].split(" ")
    review = review[0]
    return int(review.replace(',', ''));

# gets the text between the hyperlink
def getReviewLink(reviewLink):
    words = reviewLink.split(">") 
    return words[-1];

def getHyperLink(reviewLink):
    words = reviewLink.split("href=\"") 
    link = words[-1].split("\"")
    link = link[0]
    return link;

def processHotel():
    hotelName = driver.find_element_by_tag_name('h1').text
    if os.path.exists(completeDir + "\\" + hotelName) or os.path.exists(baseDir + "\\" + hotelName) :
        print ("hotel " + hotelName + " is already processed, hence skipping it")
    else:
        print("Processing hotel", hotelName)
        hotelDir = baseDir + "\\" + hotelName
        if not os.path.exists(hotelDir):
            os.makedirs(hotelDir)
        reviewHeader = driver.find_element_by_xpath("//*[@id=\"HDPR_V1\"]/div[1]/h3");
        print(reviewHeader.text)
        reviewHeader = reviewHeader.text.split(" ")
        reviews = reviewHeader[0]
        reviews = int(reviews.replace(',', ''))
        noOfPage = math.ceil(int(reviews) / 10)
        if (int(reviews) % 10) > 0:
            noOfPage += 1
        print ("Total no of pages containing review ", noOfPage)
        currentPageNo = 1
        while currentPageNo <= noOfPage:
            print ("Processing reviews in page ", currentPageNo)
            print("\n")
            fo = open(hotelDir + "\\page_" + str(currentPageNo) + ".html", "w+")
            fo.write( "<html>");
            fo.write( "<head><h3>" + hotelName + " Reviews Page " + str(currentPageNo) + "</h3></head>");
            fo.write( "<body>");
            fo.write("<br>")
            readReviews(fo)
            fo.write( "</body>");
            fo.write( "</html>");
            fo.close()
            currentPageNo += 1
            if currentPageNo <= noOfPage:
                nextPageReview = driver.find_element_by_link_text(str(currentPageNo));
                try:
                    nextPageReview.click()
                    time.sleep(1)   
                except:
                    driver.maximize_window()
                    print("Failed to click on page no ", currentPageNo, " trying to close the pop up")
                    mouse = webdriver.ActionChains(driver) 
                    moreLink = driver.find_element_by_class_name("dismiss")
                    moreLink.click()
                    print ("Trying to click page ", currentPageNo, " again")
                    mouse = webdriver.ActionChains(driver) 
                    mouse.move_to_element(nextPageReview).click().perform()
                    #nextPageReview.click()
                    time.sleep(1)
    return;



**Once we found the pages, we need to iterate over the customer reviews and then store it in utf-8 format so we can read back in pandas**

In [ ]:
def readReviews(fo):
    reviewHead = re.finditer('<span class="noQuotes">([0-9A-Za-z](.|\s)*?)</span>', driver.page_source)
    try :
        while True:
            mouse = webdriver.ActionChains(driver) 
            moreLink = driver.find_element_by_class_name("partnerRvw")
            moreLink = moreLink.find_element_by_xpath("span")
            mouse.move_to_element(moreLink).click().perform()
            time.sleep(2)
            closeLink = driver.find_element_by_class_name("xCloseGreen");
            mouse.move_to_element(closeLink).click().perform()
    except:
        pass
    custReview = driver.find_elements_by_class_name("entry")
    i = 0
    for reviewTitle in reviewHead:
        reviewTitle = reviewTitle.group(1)
        fo.write(reviewTitle.encode("utf-8"))
        fo.write("<p>")
        if custReview[i].text.encode("utf-8") and custReview[i].text.encode("utf-8").strip():
            fo.write(custReview[i].text.encode("utf-8"))
        else :
            fo.write(custReview[i+1].text.encode("utf-8"))
            i += 1
        fo.write("</p>")
        fo.write("<br>")
        i += 1
    return

** Now we will iterate over 55 hotels web pages and then get the reviews for each of the hotels from the URL **

In [ ]:
# get the list of review urls of 55 hotels
while len(hotelReviewUrlLst) < 55 :
    reviews = re.finditer('<span class="more"><a onclick="((.|\s)*?)</a></span>', driver.page_source)   
    for review in reviews:
        reviewText = review.group(1)
        reviewNum = getReviews(reviewText)
        if reviewNum >= 50 and reviewNum < 501:
            link = getHyperLink(reviewText)
            hotelReviewUrlLst.append(link); 
        if len(hotelReviewUrlLst) >= 55:
            break
    
    if len(hotelReviewUrlLst) >= 55:
        break
    else :
        try:
            print (len(hotelReviewUrlLst))
            nextLink = driver.find_element_by_link_text("Next");
            nextLink.click()
            time.sleep(2)
        except:
            nextLink = driver.find_element_by_link_text("Next");
            nextLink.click()
            time.sleep(2)
    
print ("Found 50 hotel having 50 or more customer reviews")

for link in hotelReviewUrlLst:
    link = "http://www.tripadvisor.com/" + link
    driver.get(link);
    time.sleep(10)
    try:
        processHotel()
        driver.close()
        driver = webdriver.Chrome('chromedriver.exe')
    except e:
        print ("failed to process ", link, "completely, Moving on processing next hotel")
        print ("Failed to process hotel because : " + str(e))

print ("Done processing all the hotels, closing the chrome browser")  
driver.quit()